[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2022CSharp/main?urlpath=lab%2Ftree%2FDay14.ipynb)

<h2>--- Day 14: Regolith Reservoir ---</h2><p>The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads <em>behind</em> the waterfall.</p>
<p>Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.</p>
<p>As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!</p>
<p>Fortunately, your <a href="/2018/day/17">familiarity</a> with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly <em>air</em> with structures made of <em>rock</em>.</p>
<p>Your scan traces the path of each solid rock structure and reports the <code>x,y</code> coordinates that form the shape of the path, where <code>x</code> represents distance to the right and <code>y</code> represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:</p>
<pre><code>498,4 -&gt; 498,6 -&gt; 496,6
503,4 -&gt; 502,4 -&gt; 502,9 -&gt; 494,9
</code></pre>
<p>This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from <code>498,4</code> through <code>498,6</code> and another line of rock from <code>498,6</code> through <code>496,6</code>.)</p>
<p>The sand is pouring into the cave from point <code>500,0</code>.</p>
<p>Drawing rock as <code>#</code>, air as <code>.</code>, and the source of the sand as <code>+</code>, this becomes:</p>
<pre><code>
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
</code></pre>
<p>Sand is produced <em>one unit at a time</em>, and the next unit of sand is not produced until the previous unit of sand <em>comes to rest</em>. A unit of sand is large enough to fill one tile of air in your scan.</p>
<p>A unit of sand always falls <em>down one step</em> if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally <em>one step down and to the left</em>. If that tile is blocked, the unit of sand attempts to instead move diagonally <em>one step down and to the right</em>. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand <em>comes to rest</em> and no longer moves, at which point the next unit of sand is created back at the source.</p>
<p>So, drawing sand that has come to rest as <code>o</code>, the first unit of sand simply falls straight down and then stops:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......<em>o</em>.#.
#########.
</code></pre>
<p>The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
</code></pre>
<p>After a total of five units of sand have come to rest, they form this pattern:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
</code></pre>
<p>After a total of 22 units of sand:</p>
<pre><code>......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
</code></pre>
<p>Finally, only two more units of sand can possibly come to rest:</p>
<pre><code>......+...
..........
......o...
.....ooo..
....#ooo##
...<em>o</em>#ooo#.
..###ooo#.
....oooo#.
.<em>o</em>.ooooo#.
#########.
</code></pre>
<p>Once all <code><em>24</em></code> units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with <code>~</code>:</p>
<pre><code>.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
</code></pre>
<p>Using your scan, simulate the falling sand. <em>How many units of sand come to rest before sand starts flowing into the abyss below?</em></p>

In [8]:
using System.IO;

In [9]:
var input = File.ReadAllLines("input/14.txt");

In [10]:
Dictionary<(int X, int Y), char> mapCave(string[] input)
{
    var cave = new Dictionary<(int X, int Y), char>();

    void fill((int X, int Y) start, (int X, int Y) end)
    {
        var minX = Math.Min(start.X, end.X);
        var maxX = Math.Max(start.X, end.X);
        var minY = Math.Min(start.Y, end.Y);
        var maxY = Math.Max(start.Y, end.Y);

        for (var y = minY; y <= maxY; y++)
        {
            for (var x = minX; x <= maxX; x++)
            {
                cave[(x, y)] = '#';
            }
        }
    }

    foreach (var line in input)
    {
        var points = line
            .Split(" -> ")
            .Select(point => point
                .Split(",")
                .Select(int.Parse))
            .Select(point => (point.First(), point.Last()));
        var pairs = points.Zip(points.Skip(1));

        foreach (var (start, end) in pairs)
        {
            fill(start, end);
        }
    }

    return cave;
}

In [11]:
string drawCave(Dictionary<(int X, int Y), char> cave)
{
    var minX = cave.Keys.Select(k => k.X).Min();
    var maxX = cave.Keys.Select(k => k.X).Max();
    var minY = cave.Keys.Select(k => k.Y).Min();
    var maxY = cave.Keys.Select(k => k.Y).Max();

    var map = new StringBuilder();
    for (var y = minY; y <= maxY; y++)
    {
        var line = new StringBuilder();
        for (var x = minX; x <= maxX; x++)
        {
            if (cave.ContainsKey((x, y)))
            {
                line.Append(cave[(x, y)]);
            }
            else
            {
                line.Append('.');
            }
        }
        map.AppendLine(line.ToString());
    }

    return map.ToString();
}

In [12]:
void simulateSand(Dictionary<(int X, int Y), char> cave, (int X, int Y) source, bool floor = false)
{
    // var visual = display(drawCave(cave));
    var maxY = cave.Keys.Select(k => k.Y).Max();
    
    (int X, int Y) drop((int X, int Y) prev)
    {
        var (x, y) = prev;
        
        if (y > maxY)
        {
            return floor ? (x, y) : (-1, -1);
        }

        if (!cave.ContainsKey((x, y + 1)))
        {
            return drop((x, y + 1));
        }

        if (!cave.ContainsKey((x - 1, y + 1)))
        {
            return drop((x - 1, y + 1));
        }

        if (!cave.ContainsKey((x + 1, y + 1)))
        {
            return drop((x + 1, y + 1));
        }

        return (x, y);
    }

    var nextSand = drop(source);
    while (nextSand != (-1, -1))
    {
        cave[nextSand] = 'o';
        // visual.Update(drawCave(cave));
        if (floor && nextSand == source)
        {
            break;
        }
        nextSand = drop(source);
    }
}

In [13]:
#!time
var cave = mapCave(input);
simulateSand(cave, (500, 0));
cave.Values.Where(v => v == 'o').Count()

901

Wall time: 117.6796ms

<h2 id="part2">--- Part Two ---</h2><p>You realize you misread the scan. There isn't an <span title="Endless Void is my C cover band.">endless void</span> at the bottom of the scan - there's floor, and you're standing on it!</p>
<p>You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a <code>y</code> coordinate equal to <em>two plus the highest <code>y</code> coordinate</em> of any point in your scan.</p>
<p>In the example above, the highest <code>y</code> coordinate of any point is <code>9</code>, and so the floor is at <code>y=11</code>. (This is as if your scan contained one extra rock path like <code>-infinity,11 -&gt; infinity,11</code>.) With the added floor, the example above now looks like this:</p>
<pre><code>        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
&lt;-- etc #################### etc --&gt;
</code></pre>
<p>To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at <code>500,0</code>, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after <code><em>93</em></code> units of sand come to rest:</p>
<pre><code>............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
</code></pre>
<p>Using your scan, simulate the falling sand until the source of the sand becomes blocked. <em>How many units of sand come to rest?</em></p>

In [14]:
#!time
var cave = mapCave(input);
simulateSand(cave, (500, 0), floor: true);
cave.Values.Where(v => v == 'o').Count()

24589

Wall time: 584.1075ms